# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
%%capture
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import re
import numpy as np
# nltk
import nltk
nl
# The nltk packages should be better installed systemwide as described here:
# https://www.nltk.org/data.html
# nltk.download('stopwords')
# nltk.download('wordnet') # download for lemmatization
# nltk.download('punkt')
# nltk.download('omw-1.4')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
# sklearn
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
# from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
# from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
# other models
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# pickle
import pickle

In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterData.db')
df = pd.read_sql_table('TextMessages', engine)
X = df[["message", "original", "genre"]]
Y = df.drop(columns= ["id", "message", "original", "genre"])

In [3]:
# Split the data in training and testing datasets
# X_train, X_test, y_train, y_test = train_test_split(X, Y, train_size = 0.05) # We drastically decrease the train_size to allow our GridSearch to run in a feasible amount of time

# Code testing dataset - only 100 samples
X_train, X_test, y_train, y_test = train_test_split(X[:10], Y[:10], train_size = 0.5)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # Normalization
    
    # Convert to lower case
    text = text.lower()
    
    # Remove punctuation characters - this regex finds everything which is not a combination of letters
    # and numbers and replaces it with a whitespace
    text = re.sub(r"[^a-zA-Z0-9]", " ", text)
    
    
    # Tokenization
    
    # Split into tokens
    words = word_tokenize(text)
    
    
    # Remove stopwords
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Part-of-speech tagging maybe useful here?
    # Named Entity Recognition usefuk here?
    
    # Stemming - only keep the stem of a word, simple find and replace method which removes f.e. "ing"
    # stemmed = [PorterStemmer().stem(w) for w in words]
    
    # Lemmatization - more complex appraoch using dictionaries which can f.e. map "is" and "was" to "be"
    # Lemmatize verbs by specifying pos
    lemmed_verbs = [WordNetLemmatizer().lemmatize(w, pos='v') for w in words]
    # Reduce nouns to their root form
    lemmed_nouns = [WordNetLemmatizer().lemmatize(w) for w in lemmed_verbs]
    return lemmed_nouns

In [5]:
# Calculate the average accuracy for each target column
def calculate_accuracy(name, model, y_test, y_pred, _print=True):
    columns = y_test.columns
    y_pred_df = pd.DataFrame(y_pred, columns = columns)
    accuracy = (y_pred_df == y_test.reset_index().drop(["index"], axis = 1)).mean()
    report = classification_report(y_true = y_test,
                              y_pred = y_pred,
                              target_names = list(y_test.columns),
                            #  output_dict = True,
                              zero_division = 0)
    if _print:
        print(f"F1 score, recall and precision per category {name}: \n")
        # print(f"Average accuracy: {accuracy.mean()}")
        # print(accuracy)
        print(report)
    
    return {'name' : name, 'model': model, 'report' : report}

def print_results(results):
    for result in results:
        print(result["name"])
        print(result["model"])
        print(result["report"])

def print_parameters(classifier_name, classifier):
    """
    This function prints all the available parameters, which a classifier consists off. These parameters can be used in GridSearch to optimize the classifier for our problem
    """
    print(f"Available parameters to optimize for {classifier_name}:")
    params = classifier.get_params().keys()
    print(params)

In [6]:
# Define how many cores to use for training
n_jobs = 8

In [7]:
# Create an empty array to store all the results and the models to find the best one in the end
results = []

# Native model without optimization (MultiOutputClassifier with RandomForestClassifier)

In [8]:
def fit_random_forest(results, X_train, X_test,y_train, y_test):
    # pipeline = Pipeline([
    #         ('features', FeatureUnion([
    #             ('text_pipeline', Pipeline([
    #                 ('vect', CountVectorizer(tokenizer=tokenize)),
    #                 ('tfidf', TfidfTransformer())
    #             ]))
    #         ])),
    #         ('clf', MultiOutputClassifier(RandomForestClassifier()))
    #     ])
    random_forest_pipe = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs = n_jobs)))
    ])
    random_forest_pipe.fit(X_train["message"], y_train)
    y_pred = random_forest_pipe.predict(X_test["message"])
    results.append(calculate_accuracy("MultiOutputClassifier RandomForest", random_forest_pipe, y_test, y_pred,_print = False))
    return results, random_forest_pipe

results, random_forest_pipe = fit_random_forest(results, X_train, X_test, y_train, y_test)

# kNN

In [9]:
def fit_knn(results, X_train, X_test,y_train, y_test):
    knn_pipe = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', KNeighborsClassifier(n_jobs = n_jobs))
        ])
    knn_pipe.fit(X_train["message"], y_train)
    y_pred_knn = knn_pipe.predict(X_test["message"])
    results.append(calculate_accuracy("kNN", knn_pipe, y_test, y_pred_knn,_print = False))
    return results, knn_pipe

results, knn_pipe = fit_knn(results, X_train, X_test, y_train, y_test)

# Decision tree


In [18]:
def fit_decision_tree(results, X_train, X_test,y_train, y_test):
    decision_tree_pipe = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
           ('tfidf', TfidfTransformer()),
            ('clf', DecisionTreeClassifier())
        ])
    decision_tree_pipe.fit(X_train["message"], y_train)
    y_pred_decision_tree = decision_tree_pipe.predict(X_test["message"])
    results.append(calculate_accuracy("Decision Tree", decision_tree_pipe, y_test, y_pred_decision_tree,_print = False))
    return results, decision_tree_pipe

results, decision_tree_pipe = fit_decision_tree(results, X_train, X_test, y_train, y_test)

In [19]:
print_results(results)

MultiOutputClassifier RandomForest
Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7fa76ceae9e0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier(n_jobs=8)))])
                        precision    recall  f1-score   support

               related       0.60      1.00      0.75         3
               request       0.00      0.00      0.00         1
                 offer       0.00      0.00      0.00         0
           aid_related       0.20      1.00      0.33         1
          medical_help       0.00      0.00      0.00         0
      medical_products       0.00      0.00      0.00         0
     search_and_rescue       0.00      0.00      0.00         0
              security       0.00      0.00      0.00         0
              military       0.00      0.00      0.00         0
                 water       0.00      0.00      0.00     

# Improve models using GridSearch

## MultiOutputClassifier + RandomForestClassifier

In [20]:
from math import sqrt

In [21]:
from joblib import Parallel, delayed
import joblib

In [22]:
Parallel(n_jobs)(delayed(sqrt)(i ** 2) for i in range(199))

[0.0,
 1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 30.0,
 31.0,
 32.0,
 33.0,
 34.0,
 35.0,
 36.0,
 37.0,
 38.0,
 39.0,
 40.0,
 41.0,
 42.0,
 43.0,
 44.0,
 45.0,
 46.0,
 47.0,
 48.0,
 49.0,
 50.0,
 51.0,
 52.0,
 53.0,
 54.0,
 55.0,
 56.0,
 57.0,
 58.0,
 59.0,
 60.0,
 61.0,
 62.0,
 63.0,
 64.0,
 65.0,
 66.0,
 67.0,
 68.0,
 69.0,
 70.0,
 71.0,
 72.0,
 73.0,
 74.0,
 75.0,
 76.0,
 77.0,
 78.0,
 79.0,
 80.0,
 81.0,
 82.0,
 83.0,
 84.0,
 85.0,
 86.0,
 87.0,
 88.0,
 89.0,
 90.0,
 91.0,
 92.0,
 93.0,
 94.0,
 95.0,
 96.0,
 97.0,
 98.0,
 99.0,
 100.0,
 101.0,
 102.0,
 103.0,
 104.0,
 105.0,
 106.0,
 107.0,
 108.0,
 109.0,
 110.0,
 111.0,
 112.0,
 113.0,
 114.0,
 115.0,
 116.0,
 117.0,
 118.0,
 119.0,
 120.0,
 121.0,
 122.0,
 123.0,
 124.0,
 125.0,
 126.0,
 127.0,
 128.0,
 129.0,
 130.0,
 131.0,
 132.0,
 133.0,
 134.0,
 135.0,
 136.0,
 137.0,
 138.0

In [23]:
# Using dask - maybe overkill here
# https://joblib.readthedocs.io/en/latest/auto_examples/parallel/distributed_backend_simple.html
from dask.distributed import Client
# Set up a Dask client with 4 threads and 1 worker
client = Client(processes=False, threads_per_worker=8, n_workers=1)

# # Run grid search using joblib and a Dask backend
# with joblib.parallel_backend("____"):
#     engrid.fit(x_train, y_train)

# plot_enet(*enet_path(x_test, y_test, eps=5e-5, fit_intercept=False,
#                     l1_ratio=engrid.best_params_["l1_ratio"])[:2])

In [27]:
#print_parameters("Random Forest", random_forest_pipe)
from tqdm import tqdm

def random_forest_gridsearch(results, random_forest_pipe, X_train, X_test, y_train, y_test):
    # Define a dictionary with all the parameters
    parameters_mo_rf = {
        # vect
        # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html


        # tfidf
        # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
        'tfidf__norm' : ['l1', 'l2'],
        #  'tfidf__use_idf' : [True, False],
        # 'tfidf__smooth_idf': [True, False],
        # 'tfidf__sublinear_tf' : [True, False],

        # clf
        # https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html
        'clf__estimator__criterion' : ['gini', 'entropy'],
        'clf__estimator__n_estimators': [50, 100, 150, 200],
        'clf__estimator__max_depth' : [None, 5, 10],
    }
    cv_rf = GridSearchCV(random_forest_pipe, param_grid=parameters_mo_rf, 
                         # Automatically stops GridSearch and raises occuring errors
                         error_score="raise", n_jobs = n_jobs)
    print("Pipeline created - GridSearch starts now")
    tqdm(cv_rf.fit(X_train["message"], y_train))
    print("GridSearch finished")
    # with joblib.parallel_backend("dask"):
    #     tqdm(cv_rf.fit(X_train["message"], y_train))
    # Parallel(n_jobs)(cv_rf.fit(X_train["message"], y_train))
    y_pred_mo_rf_cv = cv_rf.predict(X_test["message"])
    results.append(calculate_accuracy("MultiOutputClassifier Random Forest CV", cv_rf, y_test, y_pred_mo_rf_cv,_print = False))
    return results, cv_rf

results, cv_rf = random_forest_gridsearch(results, random_forest_pipe, X_train, X_test, y_train, y_test)

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

Task was destroyed but it is pending!
task: <Task pending name='Task-571526' coro=<DaskDistributedBackend._collect() running at /home/martin/.local/lib/python3.10/site-packages/joblib/_dask.py:202> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[IOLoop.add_future.<locals>.<lambda>() at /home/martin/.local/lib/python3.10/site-packages/tornado/ioloop.py:688]>


In [ ]:
5

## kNN

In [ ]:
print_parameters("kNN", knn_pipe)

def kNN_gridsearch(results, knn_pipe, X_train, X_test, y_train, y_test):
    parameters_knn = {
        # vect
        # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html


        # tfidf
        # https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfTransformer.html
        'tfidf__norm' : ['l1', 'l2'],
      #  'tfidf__use_idf' : [True, False],
      #  'tfidf__smooth_idf': [True, False],
      #  'tfidf__sublinear_tf' : [True, False],

        # clf
        # https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html
        'clf__n_neighbors' : [3, 5, 8],
        'clf__weights' : ['uniform', 'distance'],
        'clf__algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],

    }
    cv_knn = GridSearchCV(knn_pipe, param_grid=parameters_knn, n_jobs=n_jobs, 
                          # Automatically stops GridSearch and raises occuring errors
                          error_score="raise") 
    cv_knn.fit(X_train["message"], y_train)
    y_pred_knn_cv = cv_knn.predict(X_test["message"])
    results.append(calculate_accuracy("kNN CV", cv_knn, y_test, y_pred_knn_cv,_print = False))
    return results, cv_knn

results, cv_knn = kNN_gridsearch(results, knn_pipe, X_train, X_test, y_train, y_test)

# Classification report

In [ ]:
from sklearn.metrics import classification_report

report = classification_report(y_true = y_test,
                              y_pred = y_pred,
                              target_names = list(y_test.columns),
                              output_dict = True,
                              zero_division = 0)

# Evaluate the results

In [ ]:
print_results(results)

As we can see, the models performed all very similar. Only the decision tree model is a bit worse compared to the other ones. Surprisingly, our unoptimized orginal model with a MultiOutpuClassfier and a RandomForestClassifier performed best. Therefore we can assume that the standard model configuration fits good to our problem and the optimization attempt only leads us away from the optimum. 94% is a quite good result so we can stick with that model.

In [ ]:
best_model = results[0]['model']

Now that we found the best model configuration, we retrain the model with 80% of the data

In [ ]:
X_train_new, X_test_new, y_train_new, y_test_new = train_test_split(X, Y, train_size = 0.80)
best_model.fit(X_train_new["message"], y_train_new)
y_pred_final = best_model.predict(X_test_new["message"])

### 9. Export your model as a pickle file

In [ ]:
model_params = best_model.get_params()
model = best_model

fileObj = open('model_params.obj', 'wb')
pickle.dump(model_params,fileObj)
fileObj.close()

fileObj = open('model.obj', 'wb')
pickle.dump(model,fileObj)
fileObj.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.